<a href="https://colab.research.google.com/github/1ai-hub/prototype/blob/main/LoC_v0_phase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LoC v0 — Paper1 toxicity anchor notebook

This notebook is **Version 0** of the Liver-on-Chip (LoC) pipeline.

Goals:
- Load the LoC CSV (paper1) correctly
- Parse special values like `>300`, `<0.1`
- Build a **row-level toxicity score** and a **binary toxicity label**
- Split **by drug (GroupKFold)** to avoid donor leakage
- Train simple baselines for:
  1) Toxicity classification (toxic vs non-toxic)
  2) Mechanism label classification (`if_label`)

> You can later plug in RDKit / GNN features once the evaluation harness is stable.


In [1]:
# If running in Colab, uncomment:
!pip -q install pandas numpy scikit-learn matplotlib

import pandas as pd
import numpy as np
import re
from pathlib import Path

from sklearn.model_selection import GroupKFold
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    accuracy_score, f1_score, classification_report,
    confusion_matrix
)
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt


## 1) Load data

Place your CSV at the path below (already uploaded in this environment):
- `loc_all_drug_donor_rows_paper1.csv`

If you move the file elsewhere, update `CSV_PATH`.


In [4]:
CSV_PATH = Path("/content/loc_all_drug_donor_rows_paper1.csv")
assert CSV_PATH.exists(), f"CSV not found at {CSV_PATH}"

df_raw = pd.read_csv(CSV_PATH)
df_raw.head(), df_raw.shape


(         drug  donor albumin_ic50_cmax_mult alt_lowest_cmax_mult  \
 0   Clozapine      1                     33                  300   
 1   Clozapine      2                     91                 >300   
 2   Clozapine      3                     67                  100   
 3  Olanzapine      1                   >300                 >300   
 4  Olanzapine      2                   >300                 >300   
 
   morph_lowest_cmax_mult     if_label  
 0                    100    Apoptosis  
 1                    100    Apoptosis  
 2                    100    Apoptosis  
 3                   >300    Apoptosis  
 4                   >300  No findings  ,
 (51, 6))

## 2) Parse special numeric values (`>X`, `<X`)

Prototype rule:
- `>X` → `X` (meaning: no toxicity observed up to X; this is a *lower bound* on safety)
- `<X` → `X` (meaning: very sensitive; this is an *upper bound* on toxicity threshold)

Keep a boolean flag column per field indicating whether it was capped (optional).


In [5]:
SPECIAL_RE = re.compile(r'^\s*([<>])\s*([0-9]*\.?[0-9]+)\s*$')

def parse_cmax_mult(x):
    """Parse values like '>300', '<0.1', '30', 30.0 into float.
    Returns (value_float, flag) where flag in {'gt','lt',None}.
    """
    if pd.isna(x):
        return np.nan, None
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x), None
    s = str(x).strip()
    m = SPECIAL_RE.match(s)
    if m:
        sign, num = m.group(1), float(m.group(2))
        return num, ('gt' if sign == '>' else 'lt')
    # plain numeric string
    try:
        return float(s), None
    except ValueError:
        return np.nan, None

NUM_COLS = ["albumin_ic50_cmax_mult", "alt_lowest_cmax_mult", "morph_lowest_cmax_mult"]

df = df_raw.copy()

for col in NUM_COLS:
    parsed = df[col].apply(parse_cmax_mult)
    df[col] = parsed.apply(lambda t: t[0])
    df[col + "_bound_flag"] = parsed.apply(lambda t: t[1])

df[["drug", "donor"] + NUM_COLS + [c+"_bound_flag" for c in NUM_COLS] + ["if_label"]].head()


,drug,donor,albumin_ic50_cmax_mult,alt_lowest_cmax_mult,morph_lowest_cmax_mult,albumin_ic50_cmax_mult_bound_flag,alt_lowest_cmax_mult_bound_flag,morph_lowest_cmax_mult_bound_flag,if_label
0,Clozapine,1,33.0,300.0,100.0,None,None,None,Apoptosis
1,Clozapine,2,91.0,300.0,100.0,None,gt,None,Apoptosis
2,Clozapine,3,67.0,100.0,100.0,None,None,None,Apoptosis
3,Olanzapine,1,300.0,300.0,300.0,gt,gt,gt,Apoptosis
4,Olanzapine,2,300.0,300.0,300.0,gt,gt,gt,No findings


## 3) Create toxicity score + binary label

Row-level toxicity score:

`tox_score = min(albumin_ic50, alt_lowest, morph_lowest)`

Lower score = more toxic (toxicity at lower exposure).

Binary label (you can change threshold):
- `toxic = 1 if tox_score <= TOX_THRESHOLD`


In [6]:
TOX_THRESHOLD = 30.0  # adjust: 10, 30, 100 etc.

df["tox_score"] = df[NUM_COLS].min(axis=1)
df["toxic"] = (df["tox_score"] <= TOX_THRESHOLD).astype(int)

df[["drug","donor"] + NUM_COLS + ["tox_score","toxic","if_label"]].sort_values("tox_score").head(10)


,drug,donor,albumin_ic50_cmax_mult,alt_lowest_cmax_mult,morph_lowest_cmax_mult,tox_score,toxic,if_label
6,Fialuridine,1,0.1,300.0,300.0,0.1,1,No findings
40,Simvastatin,1,0.1,300.0,300.0,0.1,1,No findings
39,Lomitapide,1,4.0,1000.0,0.1,0.1,1,No findings
32,Benoxaprofen,1,8.0,100.0,0.1,0.1,1,Apoptosis
7,Fialuridine,2,1.0,300.0,300.0,1.0,1,Steatosis
18,Tolcapone,1,3.0,100.0,10.0,3.0,1,Mitotoxicity
12,Sitaxsentan,1,4.4,300.0,100.0,4.4,1,Mitotoxicity
41,Simvastatin,2,10.0,300.0,300.0,10.0,1,Apoptosis
46,Telithromycin,1,13.0,30.0,100.0,13.0,1,Apoptosis
28,Levofloxacin,1,15.0,15.0,15.0,15.0,1,No findings


## 4) Sanity checks (counts + leakage risk)

**Critical:** Split by `drug`, not by row.


In [7]:
print("Rows:", len(df))
print("Unique drugs:", df["drug"].nunique())
print("Donors:", sorted(df["donor"].dropna().unique().tolist()))
print("\nToxic label distribution:")
print(df["toxic"].value_counts(dropna=False))

print("\nMechanism label distribution (if_label):")
print(df["if_label"].value_counts(dropna=False))

# show drugs with multiple donors
multi = df.groupby("drug")["donor"].nunique().sort_values(ascending=False)
print("\nDrugs with multiple donors (top):")
print(multi.head(15))


Rows: 51
Unique drugs: 27
Donors: [1, 2, 3]

Toxic label distribution:
toxic
0    34
1    17
Name: count, dtype: int64

Mechanism label distribution (if_label):
if_label
No findings     22
Apoptosis       21
Mitotoxicity     6
Steatosis        2
Name: count, dtype: int64

Drugs with multiple donors (top):
drug
Ambrisentan      3
Clozapine        3
Levofloxacin     3
Trovafloxacin    3
Sitaxsentan      3
Olanzapine       3
Pioglitazone     2
FIRU             2
Fialuridine      2
Simvastatin      2
Benoxaprofen     2
Ximelagatran     2
Troglitazone     2
Tolcapone        2
Tacrine          2
Name: donor, dtype: int64


## 5) GroupKFold split by drug (evaluation harness)

We evaluate baselines with **GroupKFold** where `groups = drug`.
This prevents donor leakage.


In [8]:
X = df.drop(columns=["toxic"])  # we'll pick features per task
y = df["toxic"].values
groups = df["drug"].values

gkf = GroupKFold(n_splits=5)

splits = list(gkf.split(df, y, groups=groups))
len(splits), splits[0][0][:5], splits[0][1][:5]


(5, array([0, 1, 2, 3, 4]), array([10, 20, 25, 26, 27]))

## 6) Baseline features (v0)

Since we don't yet have SMILES/RDKit/GNN, v0 uses only:
- `donor` (categorical)
- the three numeric LoC measures

This is *not* a real predictor of unseen drugs — but it validates:
- parsing
- scoring
- splitting
- training loop

Once you add molecular features, keep the **same evaluation harness**.


In [9]:
FEATURE_COLS_NUM = NUM_COLS
FEATURE_COLS_CAT = ["donor"]

X_feat = df[FEATURE_COLS_NUM + FEATURE_COLS_CAT].copy()

# Preprocess: impute numeric, one-hot encode donor
preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median"))
        ]), FEATURE_COLS_NUM),
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), FEATURE_COLS_CAT),
    ],
    remainder="drop"
)

clf_lr = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

clf_rf = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", RandomForestClassifier(
        n_estimators=500,
        random_state=42,
        class_weight="balanced_subsample"
    ))
])

def eval_classifier(model, X, y, groups, n_splits=5):
    gkf = GroupKFold(n_splits=n_splits)
    y_true_all, y_prob_all, y_pred_all = [], [], []
    for fold, (tr, te) in enumerate(gkf.split(X, y, groups=groups), start=1):
        model.fit(X.iloc[tr], y[tr])
        prob = model.predict_proba(X.iloc[te])[:, 1]
        pred = (prob >= 0.5).astype(int)
        y_true_all.append(y[te]); y_prob_all.append(prob); y_pred_all.append(pred)
    y_true_all = np.concatenate(y_true_all)
    y_prob_all = np.concatenate(y_prob_all)
    y_pred_all = np.concatenate(y_pred_all)
    auc = roc_auc_score(y_true_all, y_prob_all)
    ap = average_precision_score(y_true_all, y_prob_all)
    acc = accuracy_score(y_true_all, y_pred_all)
    f1 = f1_score(y_true_all, y_pred_all)
    return {"roc_auc": auc, "avg_precision": ap, "accuracy": acc, "f1": f1}

print("LogReg:", eval_classifier(clf_lr, X_feat, y, groups))
print("RandForest:", eval_classifier(clf_rf, X_feat, y, groups))


LogReg: {'roc_auc': np.float64(0.9705882352941176), 'avg_precision': np.float64(0.9610534409842368), 'accuracy': 0.9019607843137255, 'f1': 0.8484848484848485}
RandForest: {'roc_auc': np.float64(0.9878892733564014), 'avg_precision': np.float64(0.9777372713791399), 'accuracy': 0.9215686274509803, 'f1': 0.8888888888888888}


## 7) Mechanism label baseline (`if_label`)

Task: predict `if_label` using the same feature set.

Notes:
- This is **multi-class** classification.
- We again split by `drug`.


In [10]:
# drop rows with missing mechanism label
df_m = df.dropna(subset=["if_label"]).copy()

X_m = df_m[FEATURE_COLS_NUM + FEATURE_COLS_CAT]
y_m = df_m["if_label"].values
groups_m = df_m["drug"].values

mech_clf = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", LogisticRegression(max_iter=3000, class_weight="balanced", multi_class="auto"))
])

def eval_multiclass(model, X, y, groups, n_splits=5):
    gkf = GroupKFold(n_splits=n_splits)
    y_true_all, y_pred_all = [], []
    for fold, (tr, te) in enumerate(gkf.split(X, y, groups=groups), start=1):
        model.fit(X.iloc[tr], y[tr])
        pred = model.predict(X.iloc[te])
        y_true_all.append(y[te]); y_pred_all.append(pred)
    y_true_all = np.concatenate(y_true_all)
    y_pred_all = np.concatenate(y_pred_all)
    return {
        "accuracy": accuracy_score(y_true_all, y_pred_all),
        "macro_f1": f1_score(y_true_all, y_pred_all, average="macro"),
        "report": classification_report(y_true_all, y_pred_all)
    }

res = eval_multiclass(mech_clf, X_m, y_m, groups_m)
print({k:v for k,v in res.items() if k != "report"})
print("\nClassification report:\n", res["report"])


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOT

{'accuracy': 0.29411764705882354, 'macro_f1': 0.20826074332171893}

Classification report:
               precision    recall  f1-score   support

   Apoptosis       0.36      0.19      0.25        21
Mitotoxicity       0.07      0.17      0.10         6
 No findings       0.53      0.45      0.49        22
   Steatosis       0.00      0.00      0.00         2

    accuracy                           0.29        51
   macro avg       0.24      0.20      0.21        51
weighted avg       0.38      0.29      0.32        51



## 8) Next steps (upgrade path)

When you’re ready to move beyond v0:

1) Add a **SMILES** column (manual map or PubChem lookup offline).
2) Build RDKit descriptors (Morgan fingerprints).
3) Replace v0 features with (fingerprints + optional donor).
4) Keep **GroupKFold by drug** exactly the same.
5) Add a mechanism head to predict `if_label` jointly with toxicity.


In [11]:
print(df.shape)
print(df.columns.tolist())
print(df.head(3))
print(df["toxic"].value_counts(dropna=False))
print("unique drugs:", df["drug"].nunique())

(51, 11)
['drug', 'donor', 'albumin_ic50_cmax_mult', 'alt_lowest_cmax_mult', 'morph_lowest_cmax_mult', 'if_label', 'albumin_ic50_cmax_mult_bound_flag', 'alt_lowest_cmax_mult_bound_flag', 'morph_lowest_cmax_mult_bound_flag', 'tox_score', 'toxic']
        drug  donor  albumin_ic50_cmax_mult  alt_lowest_cmax_mult  \
0  Clozapine      1                    33.0                 300.0   
1  Clozapine      2                    91.0                 300.0   
2  Clozapine      3                    67.0                 100.0   

   morph_lowest_cmax_mult   if_label albumin_ic50_cmax_mult_bound_flag  \
0                   100.0  Apoptosis                              None   
1                   100.0  Apoptosis                              None   
2                   100.0  Apoptosis                              None   

  alt_lowest_cmax_mult_bound_flag morph_lowest_cmax_mult_bound_flag  \
0                            None                              None   
1                              gt     

In [12]:
import numpy as np
import pandas as pd

ENDPOINTS = [
    "albumin_ic50_cmax_mult",
    "alt_lowest_cmax_mult",
    "morph_lowest_cmax_mult",
]
FLAG_COLS = [
    "albumin_ic50_cmax_mult_bound_flag",
    "alt_lowest_cmax_mult_bound_flag",
    "morph_lowest_cmax_mult_bound_flag",
]

# Ensure clean types
df = df.copy()
df["drug"] = df["drug"].astype(str).str.strip()
df["donor"] = df["donor"].astype(str)

for c in ENDPOINTS:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# --- Bias 4.1: Worst-case dominance structure ---
df["tox_score"] = df[ENDPOINTS].min(axis=1)
df["tox_driver"] = df[ENDPOINTS].idxmin(axis=1)  # which endpoint is worst

for ep in ENDPOINTS:
    df[f"{ep}_margin_to_worst"] = df[ep] - df["tox_score"]  # relative pattern

# --- Bias 4.2: Dominance indicators (1-hot later at drug level) ---
for ep in ENDPOINTS:
    df[f"driver_is_{ep}"] = (df["tox_driver"] == ep).astype(int)

# --- Bias 4.3: Severity bins (robust anchors) ---
# You can tune these later; these are sensible defaults for your scale
bins = [-np.inf, 30, 100, np.inf]
labels = [0, 1, 2]  # 0=Severe, 1=Moderate, 2=Mild (lower is worse)
for ep in ENDPOINTS:
    df[f"{ep}_severity_bin"] = pd.cut(df[ep], bins=bins, labels=labels).astype(float)

# --- Bound flags as binary (some rows have "gt"/"lt"/None)
def flag_to_binary(x):
    if pd.isna(x):
        return 0
    return 1

for fc in FLAG_COLS:
    df[f"{fc}_is_bounded"] = df[fc].apply(flag_to_binary).astype(int)

# keep toxic label (already exists) but make sure it's int
df["toxic"] = df["toxic"].astype(int)

df.head(3)

,drug,donor,albumin_ic50_cmax_mult,alt_lowest_cmax_mult,morph_lowest_cmax_mult,if_label,albumin_ic50_cmax_mult_bound_flag,alt_lowest_cmax_mult_bound_flag,morph_lowest_cmax_mult_bound_flag,tox_score,...,morph_lowest_cmax_mult_margin_to_worst,driver_is_albumin_ic50_cmax_mult,driver_is_alt_lowest_cmax_mult,driver_is_morph_lowest_cmax_mult,albumin_ic50_cmax_mult_severity_bin,alt_lowest_cmax_mult_severity_bin,morph_lowest_cmax_mult_severity_bin,albumin_ic50_cmax_mult_bound_flag_is_bounded,alt_lowest_cmax_mult_bound_flag_is_bounded,morph_lowest_cmax_mult_bound_flag_is_bounded
0,Clozapine,1,33.0,300.0,100.0,Apoptosis,None,None,None,33.0,...,67.0,1,0,0,1.0,2.0,1.0,0,0,0
1,Clozapine,2,91.0,300.0,100.0,Apoptosis,None,gt,None,91.0,...,9.0,1,0,0,1.0,2.0,1.0,0,1,0
2,Clozapine,3,67.0,100.0,100.0,Apoptosis,None,None,None,67.0,...,33.0,1,0,0,1.0,1.0,1.0,0,0,0


In [13]:
# What features do we aggregate?
row_features_num = (
    ENDPOINTS
    + ["tox_score"]
    + [f"{ep}_margin_to_worst" for ep in ENDPOINTS]
    + [f"{ep}_severity_bin" for ep in ENDPOINTS]
    + [f"{fc}_is_bounded" for fc in FLAG_COLS]
    + [f"driver_is_{ep}" for ep in ENDPOINTS]
)

# Aggregation functions
agg_map = {}
for c in row_features_num:
    if c.startswith("driver_is_") or c.endswith("_is_bounded"):
        # proportions make more sense than mean (mean==fraction since 0/1)
        agg_map[c] = ["mean"]
    elif c.endswith("_severity_bin"):
        agg_map[c] = ["mean", "min", "max"]
    else:
        agg_map[c] = ["mean", "min", "max", "std"]

drug_df = df.groupby("drug").agg(agg_map)
drug_df.columns = ["__".join(col).strip() for col in drug_df.columns]
drug_df = drug_df.reset_index()

# Donor consistency features (VERY important)
donor_stats = df.groupby("drug").agg(
    n_donors=("donor", "nunique"),
    frac_donors_toxic=("toxic", "mean"),
    any_toxic=("toxic", "max"),
    all_toxic=("toxic", "min"),
)
donor_stats = donor_stats.reset_index()

drug_df = drug_df.merge(donor_stats, on="drug", how="left")

# Drug-level targets
# Toxic: any donor toxic (conservative); you can also try majority later
y_tox = donor_stats.set_index("drug")["any_toxic"].astype(int).reindex(drug_df["drug"]).values

# Mechanism: if multiple donors have labels, take the mode; if none -> missing
def mode_or_nan(s):
    s = s.dropna().astype(str)
    if len(s) == 0:
        return np.nan
    return s.value_counts().idxmax()

drug_mech = df.groupby("drug")["if_label"].apply(mode_or_nan).reset_index()
drug_df = drug_df.merge(drug_mech, on="drug", how="left")

print("Drug-level table shape:", drug_df.shape)
print("n drugs:", drug_df["drug"].nunique())
print("tox class balance:", pd.Series(y_tox).value_counts().to_dict())
print("mech labeled drugs:", drug_df["if_label"].notna().sum())
drug_df.head(3)


Drug-level table shape: (27, 49)
n drugs: 27
tox class balance: {0: 14, 1: 13}
mech labeled drugs: 27


,drug,albumin_ic50_cmax_mult__mean,albumin_ic50_cmax_mult__min,albumin_ic50_cmax_mult__max,albumin_ic50_cmax_mult__std,alt_lowest_cmax_mult__mean,alt_lowest_cmax_mult__min,alt_lowest_cmax_mult__max,alt_lowest_cmax_mult__std,morph_lowest_cmax_mult__mean,...,alt_lowest_cmax_mult_bound_flag_is_bounded__mean,morph_lowest_cmax_mult_bound_flag_is_bounded__mean,driver_is_albumin_ic50_cmax_mult__mean,driver_is_alt_lowest_cmax_mult__mean,driver_is_morph_lowest_cmax_mult__mean,n_donors,frac_donors_toxic,any_toxic,all_toxic,if_label
0,Ambrisentan,533.333333,300.0,1000.0,404.145188,533.333333,300.0,1000.0,404.145188,533.333333,...,1.0,1.0,1.0,0.0,0.0,3,0.0,0,0,No findings
1,Asunaprevir,190.000000,190.0,190.0,NaN,1000.000000,1000.0,1000.0,NaN,1000.000000,...,1.0,0.0,1.0,0.0,0.0,1,0.0,0,0,Apoptosis
2,Benoxaprofen,14.000000,8.0,20.0,8.485281,100.000000,100.0,100.0,0.000000,50.050000,...,0.5,0.5,0.5,0.0,0.5,2,1.0,1,1,Apoptosis


In [15]:
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from scipy.stats import spearmanr

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Build X
FEATURE_COLS = [c for c in drug_df.columns if c not in ["drug", "if_label"]]
X = drug_df[FEATURE_COLS].astype(float).values
groups = drug_df["drug"].astype(str).values  # each row is a drug; groups still drug for clarity

# Check NaNs
nan_counts = pd.Series(X.flatten()).isna().sum()
print("Total NaNs in X:", nan_counts)

# Column-wise NaN count
nan_per_col = pd.Series(np.isnan(X).sum(axis=0), index=FEATURE_COLS)
print("NaNs per feature (non-zero only):")
print(nan_per_col[nan_per_col > 0])

# --- Impute NaNs with column median (drug-level, safe) ---
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
X = imputer.fit_transform(X)

print("NaNs after imputation:", np.isnan(X).sum())

# Toxic target
y_tox = y_tox.astype(int)

# Mechanism target (masked)
has_mech = drug_df["if_label"].notna().values
le = LabelEncoder()
y_mech = np.full(len(drug_df), -1, dtype=int)
if has_mech.any():
    y_mech[has_mech] = le.fit_transform(drug_df.loc[has_mech, "if_label"].astype(str).values)
    n_mech = len(le.classes_)
else:
    n_mech = 0

print("X:", X.shape, "tox:", y_tox.shape, "mech labeled:", int(has_mech.sum()), "mech classes:", n_mech)

class MultiTaskMLP(nn.Module):
    def __init__(self, in_dim, hidden, n_mech):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(0.25),
        )
        self.tox_head = nn.Linear(hidden, 1)
        self.n_mech = n_mech
        self.mech_head = nn.Linear(hidden, n_mech) if n_mech > 0 else None

    def forward(self, x):
        z = self.encoder(x)
        tox_logit = self.tox_head(z).squeeze(-1)
        mech_logits = self.mech_head(z) if self.mech_head is not None else None
        return tox_logit, mech_logits

def eval_ranking(y_true_tox, tox_prob, true_tox_severity=None):
    # Ranking stability metric: Spearman corr between predicted prob and a severity signal
    # If you have severity proxy (e.g., min tox_score), use it; else correlate with label.
    if true_tox_severity is None:
        rho = spearmanr(tox_prob, y_true_tox).correlation
    else:
        rho = spearmanr(tox_prob, true_tox_severity).correlation
    return rho

def train_eval_one_model(X, y_tox, y_mech, has_mech, groups, n_mech,
                         n_splits=5, hidden=64, lr=2e-3, epochs=400, batch_size=16,
                         mech_w=0.6, seed=42):

    np.random.seed(seed)
    torch.manual_seed(seed)

    gkf = GroupKFold(n_splits=n_splits)
    rows = []

    # severity proxy for ranking: worst-case (min) tox_score at drug level
    # lower tox_score = worse; convert to "worse" score by negating
    severity_proxy = -drug_df["tox_score__min"].values if "tox_score__min" in drug_df.columns else None

    for fold, (tr, te) in enumerate(gkf.split(X, y_tox, groups=groups), start=1):
        scaler = StandardScaler()
        Xtr = scaler.fit_transform(X[tr])
        Xte = scaler.transform(X[te])

        Xtr_t = torch.tensor(Xtr, dtype=torch.float32)
        Xte_t = torch.tensor(Xte, dtype=torch.float32)
        ytox_tr = torch.tensor(y_tox[tr], dtype=torch.float32)
        ytox_te = y_tox[te]

        ymech_tr = torch.tensor(y_mech[tr], dtype=torch.long)
        mechmask_tr = torch.tensor(has_mech[tr].astype(np.float32), dtype=torch.float32)

        ds = TensorDataset(Xtr_t, ytox_tr, ymech_tr, mechmask_tr)
        dl = DataLoader(ds, batch_size=batch_size, shuffle=True)

        model = MultiTaskMLP(in_dim=X.shape[1], hidden=hidden, n_mech=n_mech)
        opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
        bce = nn.BCEWithLogitsLoss()
        ce = nn.CrossEntropyLoss(reduction="none")

        model.train()
        for _ in range(epochs):
            for xb, yb_tox, yb_mech, mb in dl:
                opt.zero_grad()
                tox_logit, mech_logits = model(xb)

                loss_tox = bce(tox_logit, yb_tox)

                if n_mech > 0:
                    per = ce(mech_logits, yb_mech)
                    loss_mech = (per * mb).sum() / (mb.sum() + 1e-8)
                    loss = loss_tox + mech_w * loss_mech
                else:
                    loss = loss_tox

                loss.backward()
                opt.step()

        model.eval()
        with torch.no_grad():
            tox_logit_te, mech_logits_te = model(Xte_t)
            tox_prob = torch.sigmoid(tox_logit_te).cpu().numpy()
            tox_pred = (tox_prob >= 0.5).astype(int)

        # Metrics
        auc = roc_auc_score(ytox_te, tox_prob) if len(np.unique(ytox_te)) > 1 else np.nan
        f1 = f1_score(ytox_te, tox_pred, zero_division=0)
        acc = accuracy_score(ytox_te, tox_pred)

        rho = eval_ranking(ytox_te, tox_prob, None if severity_proxy is None else severity_proxy[te])

        out = {"fold": fold, "auc": auc, "f1": f1, "acc": acc, "rank_spearman": rho,
               "n_test_drugs": len(te), "pos_test": int(ytox_te.sum())}

        # Mechanism macro-F1 on labeled test drugs only
        if n_mech > 0:
            te_has = has_mech[te]
            if te_has.any():
                mech_pred = mech_logits_te.argmax(dim=1).cpu().numpy()
                out["mech_macro_f1"] = f1_score(y_mech[te][te_has], mech_pred[te_has],
                                                average="macro", zero_division=0)
            else:
                out["mech_macro_f1"] = np.nan

        rows.append(out)

    return pd.DataFrame(rows)

metrics_df = train_eval_one_model(X, y_tox, y_mech, has_mech, groups, n_mech)
metrics_df


Total NaNs in X: 63
NaNs per feature (non-zero only):
albumin_ic50_cmax_mult__std                    9
alt_lowest_cmax_mult__std                      9
morph_lowest_cmax_mult__std                    9
tox_score__std                                 9
albumin_ic50_cmax_mult_margin_to_worst__std    9
alt_lowest_cmax_mult_margin_to_worst__std      9
morph_lowest_cmax_mult_margin_to_worst__std    9
dtype: int64
NaNs after imputation: 0
X: (27, 47) tox: (27,) mech labeled: 27 mech classes: 3


,fold,auc,f1,acc,rank_spearman,n_test_drugs,pos_test,mech_macro_f1
0,1,1.0,0.8,0.833333,0.942857,6,2,0.625000
1,2,1.0,0.8,0.833333,0.880406,6,3,1.000000
2,3,1.0,1.0,1.000000,0.900000,5,1,0.300000
3,4,1.0,0.8,0.800000,0.894737,5,3,0.555556
4,5,1.0,1.0,1.000000,0.763158,5,4,0.600000


In [16]:
# === FINAL SETUP (helpers + model + artifact saving) ===
import numpy as np
import pandas as pd
import pickle
from pathlib import Path

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

import torch
import torch.nn as nn

# Assumes you already created:
#   df  (raw donor-level rows)
#   drug_df (drug-level aggregated table with "drug" and "if_label")
#   FEATURE_COLS (all feature columns in drug_df except ["drug","if_label"])
# If not, run the Phase-1 cells we wrote earlier to produce those.

ARTIFACT_DIR = Path("loc_v0_artifacts")
ARTIFACT_DIR.mkdir(exist_ok=True)

class MultiTaskMLP(nn.Module):
    def __init__(self, in_dim, hidden, n_mech):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(0.25),
        )
        self.tox_head = nn.Linear(hidden, 1)
        self.n_mech = n_mech
        self.mech_head = nn.Linear(hidden, n_mech) if n_mech > 0 else None

    def forward(self, x):
        z = self.encoder(x)
        tox_logit = self.tox_head(z).squeeze(-1)
        mech_logits = self.mech_head(z) if self.mech_head is not None else None
        return tox_logit, mech_logits

def save_pickle(obj, path: Path):
    with open(path, "wb") as f:
        pickle.dump(obj, f)

def load_pickle(path: Path):
    with open(path, "rb") as f:
        return pickle.load(f)

In [17]:
# === FINAL TRAINING CELL (train one final model on ALL 27 drugs) ===

# Build X (drug-level), tox target, mech target (masked)
FEATURE_COLS = [c for c in drug_df.columns if c not in ["drug", "if_label"]]
X_raw = drug_df[FEATURE_COLS].astype(float).values

# Toxic target at drug level:
# If you already computed y_tox earlier, keep using it.
# Else compute it from your donor-level df: "any donor toxic" per drug.
drug_any_toxic = df.groupby("drug")["toxic"].max().reindex(drug_df["drug"]).astype(int).values
y_tox = drug_any_toxic

# Mechanism labels at drug level
has_mech = drug_df["if_label"].notna().values
le = LabelEncoder()
y_mech = np.full(len(drug_df), -1, dtype=int)
if has_mech.any():
    y_mech[has_mech] = le.fit_transform(drug_df.loc[has_mech, "if_label"].astype(str).values)
    n_mech = len(le.classes_)
else:
    n_mech = 0

print("Training on drugs:", len(drug_df), "| features:", X_raw.shape[1], "| mech classes:", n_mech)

# Impute + scale (fit on ALL data for final deployed model)
imputer = SimpleImputer(strategy="median")
X_imp = imputer.fit_transform(X_raw)

scaler = StandardScaler()
X = scaler.fit_transform(X_imp)

# Torch tensors
X_t = torch.tensor(X, dtype=torch.float32)
ytox_t = torch.tensor(y_tox, dtype=torch.float32)
ymech_t = torch.tensor(y_mech, dtype=torch.long)
mechmask_t = torch.tensor(has_mech.astype(np.float32), dtype=torch.float32)

# Train
hidden = 64
lr = 2e-3
epochs = 600
mech_w = 0.6

model = MultiTaskMLP(in_dim=X.shape[1], hidden=hidden, n_mech=n_mech)
opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
bce = nn.BCEWithLogitsLoss()
ce = nn.CrossEntropyLoss(reduction="none")

model.train()
for ep in range(epochs):
    opt.zero_grad()
    tox_logit, mech_logits = model(X_t)

    loss_tox = bce(tox_logit, ytox_t)

    if n_mech > 0:
        per = ce(mech_logits, ymech_t)
        loss_mech = (per * mechmask_t).sum() / (mechmask_t.sum() + 1e-8)
        loss = loss_tox + mech_w * loss_mech
    else:
        loss = loss_tox

    loss.backward()
    opt.step()

    if (ep + 1) % 100 == 0:
        print(f"epoch {ep+1}/{epochs} | loss={loss.item():.4f} | tox={loss_tox.item():.4f}"
              + (f" | mech={loss_mech.item():.4f}" if n_mech > 0 else ""))

# Save artifacts
torch.save(
    {
        "state_dict": model.state_dict(),
        "in_dim": X.shape[1],
        "hidden": hidden,
        "n_mech": n_mech,
        "feature_cols": FEATURE_COLS,
    },
    ARTIFACT_DIR / "model.pt",
)

save_pickle(imputer, ARTIFACT_DIR / "imputer.pkl")
save_pickle(scaler, ARTIFACT_DIR / "scaler.pkl")
save_pickle(le, ARTIFACT_DIR / "mech_label_encoder.pkl")
save_pickle(
    {
        "tox_threshold": 0.5,
        "notes": "Final model trained on all available LoC drugs (drug-level aggregation).",
    },
    ARTIFACT_DIR / "meta.pkl",
)

print("✅ Saved to:", ARTIFACT_DIR.resolve())


Training on drugs: 27 | features: 47 | mech classes: 3
epoch 100/600 | loss=0.0758 | tox=0.0041 | mech=0.1195
epoch 200/600 | loss=0.0099 | tox=0.0014 | mech=0.0143
epoch 300/600 | loss=0.0014 | tox=0.0004 | mech=0.0017
epoch 400/600 | loss=0.0014 | tox=0.0002 | mech=0.0020
epoch 500/600 | loss=0.0034 | tox=0.0024 | mech=0.0016
epoch 600/600 | loss=0.0004 | tox=0.0001 | mech=0.0005
✅ Saved to: /content/loc_v0_artifacts


In [18]:
# === INFERENCE CELL (produce a usable prediction report) ===

def load_final_model(artifact_dir=ARTIFACT_DIR):
    ckpt = torch.load(Path(artifact_dir) / "model.pt", map_location="cpu")
    model = MultiTaskMLP(ckpt["in_dim"], ckpt["hidden"], ckpt["n_mech"])
    model.load_state_dict(ckpt["state_dict"])
    model.eval()

    imputer = load_pickle(Path(artifact_dir) / "imputer.pkl")
    scaler = load_pickle(Path(artifact_dir) / "scaler.pkl")
    le = load_pickle(Path(artifact_dir) / "mech_label_encoder.pkl")
    meta = load_pickle(Path(artifact_dir) / "meta.pkl")
    feature_cols = ckpt["feature_cols"]

    return model, imputer, scaler, le, meta, feature_cols

def predict_drug_table(drug_level_df: pd.DataFrame, artifact_dir=ARTIFACT_DIR):
    """
    drug_level_df must have:
      - column 'drug'
      - same FEATURE_COLS used in training (stored in artifacts)
      - optional 'if_label' (not required for inference)
    """
    model, imputer, scaler, le, meta, feature_cols = load_final_model(artifact_dir)

    X_raw = drug_level_df[feature_cols].astype(float).values
    X_imp = imputer.transform(X_raw)
    X = scaler.transform(X_imp)

    with torch.no_grad():
        tox_logit, mech_logits = model(torch.tensor(X, dtype=torch.float32))
        tox_prob = torch.sigmoid(tox_logit).cpu().numpy()

        if mech_logits is not None:
            mech_prob = torch.softmax(mech_logits, dim=1).cpu().numpy()
            mech_idx = mech_prob.argmax(axis=1)
            mech_label = le.inverse_transform(mech_idx)
            mech_conf = mech_prob.max(axis=1)
        else:
            mech_label = np.array(["NA"] * len(drug_level_df))
            mech_conf = np.array([np.nan] * len(drug_level_df))

    # Confidence heuristic (LoC-aware): more donors + more agreement => higher confidence
    # These columns exist from our aggregation cell: n_donors, frac_donors_toxic, all_toxic
    conf = []
    for _, r in drug_level_df.iterrows():
        nd = float(r.get("n_donors", 1))
        frac = float(r.get("frac_donors_toxic", 0.5))
        all_tox = float(r.get("all_toxic", 0))
        # simple bounded score
        c = 0.4
        c += 0.2 * min(nd / 3.0, 1.0)            # more donors -> higher
        c += 0.2 * (1.0 - abs(frac - 1.0))       # closer to 1 -> higher
        c += 0.2 * all_tox                       # if all donors toxic -> higher
        conf.append(float(np.clip(c, 0.0, 1.0)))

    out = pd.DataFrame({
        "drug": drug_level_df["drug"].astype(str).values,
        "tox_prob": tox_prob,
        "tox_pred": (tox_prob >= meta.get("tox_threshold", 0.5)).astype(int),
        "prediction_confidence": conf,
        "dominant_mechanism_pred": mech_label,
        "mechanism_confidence": mech_conf,
    })

    # Add ranking / percentile within this batch
    out["tox_rank"] = out["tox_prob"].rank(ascending=False, method="average").astype(int)
    out["tox_percentile"] = out["tox_prob"].rank(pct=True)  # higher = more toxic
    out = out.sort_values("tox_prob", ascending=False).reset_index(drop=True)
    return out

# ---- Example: run inference on your current 27 drugs (drug_df) ----
report = predict_drug_table(drug_df, ARTIFACT_DIR)
report.head(10)


,drug,tox_prob,tox_pred,prediction_confidence,dominant_mechanism_pred,mechanism_confidence,tox_rank,tox_percentile
0,Sitaxsentan,1.000000,1,0.666667,Mitotoxicity,0.999998,1,0.981481
1,Tolcapone,1.000000,1,0.933333,Mitotoxicity,0.999794,1,0.981481
2,Telithromycin,1.000000,1,0.866667,Apoptosis,0.999995,3,0.907407
3,Ximelagatran,1.000000,1,0.633333,No findings,0.999999,3,0.907407
4,Tacrine,1.000000,1,0.633333,Apoptosis,1.000000,5,0.833333
5,Lomitapide,1.000000,1,0.866667,No findings,1.000000,5,0.833333
6,Benoxaprofen,0.999998,1,0.933333,Apoptosis,0.999983,7,0.777778
7,Fialuridine,0.999997,1,0.933333,No findings,1.000000,8,0.740741
8,Simvastatin,0.999997,1,0.933333,No findings,0.999999,9,0.703704
9,Levofloxacin,0.999987,1,0.666667,No findings,0.999986,10,0.666667


In [19]:
from pathlib import Path

OUT_DIR = Path("/content")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Canonical drug-level table (27 rows)
drug_csv = OUT_DIR / "loc_phase1_drug_table.csv"
drug_df.to_csv(drug_csv, index=False)
print("Saved:", drug_csv)

# 2) Phase-1 CV metrics table
metrics_csv = OUT_DIR / "loc_phase1_metrics.csv"
metrics_df.to_csv(metrics_csv, index=False)
print("Saved:", metrics_csv)

# Optional: save donor-level enriched df too (if you want)
# df.to_csv(OUT_DIR / "loc_phase1_donor_table_enriched.csv", index=False)

!ls -lh /content | sed -n '1,120p'

Saved: /content/loc_phase1_drug_table.csv
Saved: /content/loc_phase1_metrics.csv
total 28K
-rw-r--r-- 1 root root 2.0K Feb  4 06:03 loc_all_drug_donor_rows_paper1.csv
-rw-r--r-- 1 root root 9.2K Feb  4 07:43 loc_phase1_drug_table.csv
-rw-r--r-- 1 root root  317 Feb  4 07:43 loc_phase1_metrics.csv
drwxr-xr-x 2 root root 4.0K Feb  4 07:20 loc_v0_artifacts
drwxr-xr-x 1 root root 4.0K Jan 16 14:24 sample_data


In [20]:
from pathlib import Path

ART_DIR = Path("/content/loc_v0_artifacts")
assert ART_DIR.exists(), f"Not found: {ART_DIR} (check folder name in /content)"

ZIP_PATH = Path("/content/loc_v0_artifacts.zip")

# Create zip
!rm -f /content/loc_v0_artifacts.zip
!zip -r /content/loc_v0_artifacts.zip /content/loc_v0_artifacts > /dev/null

print("Created:", ZIP_PATH)
!ls -lh /content/loc_v0_artifacts.zip

Created: /content/loc_v0_artifacts.zip
-rw-r--r-- 1 root root 33K Feb  4 07:43 /content/loc_v0_artifacts.zip


In [21]:
from google.colab import files

files.download("/content/loc_phase1_drug_table.csv")
files.download("/content/loc_phase1_metrics.csv")
files.download("/content/loc_v0_artifacts.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>